Notebook for generating specific instace. Useful for e.g. writing an mps file which different scripts etc. can access to train and/or test on.

In [ ]:
import ecole
from retro_branching.environments import EcoleConfiguring, EcoleBranching



env = EcoleBranching()
env.seed(0)

def gen_instance(instances, env, min_difficulty=None, max_difficulty=None, max_attempts=10000):
    '''Generates instance which can return an observation which is not None on env.reset() (i.e. not pre-solved).'''
    if min_difficulty is not None or max_difficulty is not None:
        # init env for evaluating instance difficulty
        scip_env = EcoleConfiguring()
    
    counter = 1
    while True:
        print('Attempts: {}'.format(counter))
        instance = next(instances)
        obs, action_set, reward, done, info = env.reset(instance)
        if obs is not None:
            if min_difficulty is not None or max_difficulty is not None:
                if min_difficulty is None:
                    min_difficulty = -float('inf')
                if max_difficulty is None:
                    max_difficulty = float('inf')
                # check difficulty using scip solver
                scip_env.reset(instance)
                _, _, _, _, scip_info = scip_env.step({})
                if scip_info['num_nodes'] <= max_difficulty and scip_info['num_nodes'] >= min_difficulty:
                    return instance
            else:                    
                return instance
        counter += 1
        if counter > max_attempts:
            raise Exception('Unable to generate valid instance after {} attempts.'.format(max_attempts))


In [ ]:
# instances = ecole.instance.SetCoverGenerator(n_rows=100, n_cols=100, density=0.05)
instances = ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05)

env = EcoleBranching()
env.seed(0)

instance = gen_instance(instances,
                        env,
                        min_difficulty=None,
                        max_difficulty=1)

In [ ]:
# write instance to mps file
# instance.write_problem('instance_nrows100_ncols100_density005.mps')
instance.write_problem('instance_nrows500_ncols1000_density005.mps')

In [ ]:
print(type(instance))
print(type(instances))

In [ ]:
print(type(instance) == ecole.core.scip.Model)